# Cherwell council data analysis

In [1]:
import requests

In [2]:
req = requests.get("https://www.cherwell.gov.uk/downloads/download/348/open-data")
req.raise_for_status()

Find `<a>` tags with class="download"

In [5]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text)

In [8]:
all_links = soup.select("a.download")
len(all_links)

67

In [10]:
all_links[0].get_text()

'Spend Report August 2020File type: CSVSize: 92.91 KBDownload'

In [12]:
spend_tags = [link for link in all_links if link.get_text().lower().startswith("spend")]
len(spend_tags)

45

Test single file download

In [17]:
tag = spend_tags[0]

file_name = tag["href"].split("/")[-1]

with open(file_name, 'wb') as f:
    req = requests.get(tag["href"])
    req.raise_for_status()
    f.write(req.content)

For each CSV or XLS file, download it locally

In [18]:
for tag in spend_tags:
    file_name = tag["href"].split("/")[-1]
    print(file_name)

    with open(file_name, 'wb') as f:
        req = requests.get(tag["href"])
        req.raise_for_status()
        f.write(req.content)

print("Done")

spend-report-august-2020.csv
spend-report-july-2020.csv
spending-report-june-2020.xls
spending-report-may-2020.xls
spending-report-april-2020.xls
spending-report-march-2020.xls
spend-report-february-2020.xls
spend-report-january-2020.xls
spend-report-december-2019.xls
spend-report-november-2019.xls
spending-report-september-to-october-2019.xls
spending-report-june-to-august-2019.xls
spending-report-may-19.xls
spend-over-500-april-2019.xls
spend-over-500-march-2019.xls
spend-over-500-october-2018--february-2019.xlsx
spend-over-500-september-2018.xlsx
spend-over-500-august-2018.xls
spend-over-500-july-2018.xls
spend-over-500-june-2018.xls
spend-over-500-may-2018.xls
spend-over-500-april-2018.xls
spend-over-500-march-2018.xls
spend-over-500-february-2018.xls
spend-over-500-january-2018.xls
spend-over-500-december-2017.xls
spend-over-500-november-2017.xls
spend-over-500-october-2017.xls
spend-over-500-april---september-2017.xls
spend-over-500-february---march-2017.xls
spend-over-500-januar

Now read the files and put them into one big DataFrame

In [19]:
import pandas as pd

In [24]:
df = pd.read_csv("data/spend-report-august-2020.csv", encoding="latin-1")

print(df.columns)

Index(['Date', 'Transaction Number', 'Net Amount', 'Supplier Name',
       'Service Label    ', 'Service Division Label', 'Organisational Unit',
       'Expenditure Category', 'CIPFA Detailed Expenditure Type', 'Narrative'],
      dtype='object')


In [30]:
col_list = ['Date', 'Transaction Number', 'Net Amount', 'Supplier Name',
            'Service Label', 'Service Division Label', 'Organisational Unit',
            'Expenditure Category', 'CIPFA Detailed Expenditure Type', 'Narrative']

filename = "data/spending-report-june-to-august-2019.xls"

extension = filename.split(".")[-1]

df = None

# handle all file types nicely
if extension == "csv":
    df = pd.read_csv(filename, encoding="latin-1")
elif extension == "xls":
    df = pd.read_excel(filename)
elif extension == "xlsx":
    df = pd.read_excel(filename, engine="pyopenxl")
else:
    print(f"How did we get here? {filename}")

# remove trailing spaces
df.columns = [col.strip() for col in df.columns]
    
df = df[col_list]
df.head()

,Date,Transaction Number,Net Amount,Supplier Name,Service Label,Service Division Label,Organisational Unit,Expenditure Category,CIPFA Detailed Expenditure Type,Narrative
0,2019-06-03,C034360,500.00,Cotswolds Conservation Board,Strategy and Commissioning,Strategic Planning Economy,Development Management / Development Management,Pension Contribution - Employers,Employees,NaN
1,2019-06-03,C034366,10943.59,Atkins Ltd,Commercial Development Capital,Capital Regen and Housing,Regeneration & Housing / Delivery Team,Contractors Capital Payments,Capital Works,Fairway Methodist Site \t\t\t\r\n\t\t\t\r\n 60...
2,2019-06-03,C034370,1915.30,New Recruits Professional Services Ltd,Operations and Delivery,Environmental Services,Environmental Services / Environmental Service...,Agency Staff,Employees,Annual Purchase order\r\nProvision of Agency S...
3,2019-06-03,C034383,2172.50,BRG Interim Solutions Ltd,Strategy and Commissioning,Development Management,Finance & Procurement / Head of Finance & Proc...,Agency Staff,Employees,REDACTED
4,2019-06-03,C034384,2478.65,New Recruits Professional Services Ltd,Operations and Delivery,Environmental Services,Environmental Services / Environmental Service...,Agency Staff,Employees,Annual Purchase order\r\nProvision of Agency S...


In [36]:
import os

# assume everything has these columns somewhere (maybe in a different order)
col_list = ['Date', 'Transaction Number', 'Net Amount', 'Supplier Name',
            'Service Label', 'Service Division Label', 'Organisational Unit',
            'Expenditure Category', 'CIPFA Detailed Expenditure Type', 'Narrative']

all_files = os.listdir("data")

dataframes = []

for filename in all_files:
    print(filename)
    filename = os.path.join("data", filename)
    extension = filename.split(".")[-1]

    df = None

    # handle all file types nicely
    if extension == "csv":
        df = pd.read_csv(filename, encoding="latin-1")
    elif extension == "xls":
        df = pd.read_excel(filename)
    elif extension == "xlsx":
        df = pd.read_excel(filename, engine="openpyxl")
    else:
        print(f"How did we get here? {filename}")

    # remove trailing spaces
    df.columns = [str(col).strip() for col in df.columns]

    try:
        # can we extract the columns as seen in the latest file?
        df = df[col_list]
        dataframes.append(df)
    except:
        # if not, ignore
        print("\tColumns not found, ignoring...")
        pass

print("Done")

spend-over-500-apr-12---mar13-creditors.xls
	Columns not found, ignoring...
spend-over-500-apr-13-to-feb-14-creditors.xls
	Columns not found, ignoring...
spend-over-500-apr-13-to-oct13-expenses.xls
	Columns not found, ignoring...
spend-over-500-apr-14-to-mar-15-creditors.xls
	Columns not found, ignoring...
spend-over-500-april---september-2017.xls
spend-over-500-april-2018.xls
spend-over-500-april-2019.xls
spend-over-500-april-to-june-2015-creditors.xls
	Columns not found, ignoring...
spend-over-500-august-2015.xls
	Columns not found, ignoring...
spend-over-500-august-2018.xls
spend-over-500-december-2015.xls
	Columns not found, ignoring...
spend-over-500-december-2017.xls
spend-over-500-february---march-2017.xls
spend-over-500-february-2016.xls
	Columns not found, ignoring...
spend-over-500-february-2018.xls
spend-over-500-january-2016.xls
	Columns not found, ignoring...
spend-over-500-january-2017.xlsx
spend-over-500-january-2018.xls
spend-over-500-july-2015.xls
	Columns not found, i

In [37]:
len(dataframes)

32

In [38]:
df_all = pd.concat(dataframes, ignore_index=True)
print(df_all.shape)
df_all.head()

(34817, 10)


,Date,Transaction Number,Net Amount,Supplier Name,Service Label,Service Division Label,Organisational Unit,Expenditure Category,CIPFA Detailed Expenditure Type,Narrative
0,2017-04-06 00:00:00,C010219,2925.22,New Recruits Professional Services Ltd,Operations and Delivery,Environmental Services,Environmental Services,Agency Staff,Employees,Agency Staff W/C 20/03/17
1,2017-04-06 00:00:00,C010220,668.22,New Recruits Professional Services Ltd,Operations and Delivery,Environmental Services,Environmental Services,Agency Staff,Employees,Agency Staff W/C 20/03/17
2,2017-04-06 00:00:00,C009848,996.09,Heathrow Truck Centre Ltd,Operations and Delivery,Environmental Services,Finance & Procurement,Vehicle Repair & Maintenance,Transport Related Expenditure,Annual order for 2016-2017
3,2017-04-06 00:00:00,C009417,988.00,Shelter Training & Publications,Commercial Development,Regeneration and Housing,Regeneration & Housing,Training,Employees,Arrears Management: The impact of universal cr...
4,2017-04-06 00:00:00,C010288,2000.00,The Mill Arts Centre,Operations and Delivery,Community Services,Finance & Procurement,Activities (workshop fees),Supplies and Services,Arts for Older People


In [39]:
df_all.to_csv("all_transactions.csv.gz", index=False)